In [22]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
%cd '/content/drive/My Drive/Colab Notebooks/2-parametro-ventas-insumos'

/content/drive/My Drive/Colab Notebooks/2-parametro-ventas-insumos


In [25]:
import os
import random as rd
import numpy as np
import pandas as pd

In [26]:
ruta_iniciales = os.path.join(os.getcwd(), 'archivos-iniciales')
ruta_resultados = os.path.join(os.getcwd(), 'archivos-resultados')

In [27]:
archivos_iniciales = os.listdir(ruta_iniciales)
archivos_iniciales

['reps-ids-cacao.csv',
 'reps-ids-maiz.csv',
 'reps-ids-arroz.csv',
 'arroz.csv',
 'maiz.csv',
 'cacao.csv']

IDS UNICOS

In [28]:
cacao = pd.read_csv(os.path.join(ruta_iniciales, 'cacao.csv'))

IDS REPETIDOS

In [29]:
cacao_reps = pd.read_csv(os.path.join(ruta_iniciales, 'reps-ids-cacao.csv'))

<hr>

transitorios:
  > [arroz, maiz]

permanentes
  > [cacao]

'Ventas' para arroz y maiz

'ventas' para cacao

### CORRIGIENDO LAS UNIDADES

FUNCIONES

In [30]:
def conversion_tonelada_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 4
  df[columna_valor][mascara] = df[columna_valor][mascara] * 2831.6846592
  return df  

def conversion_quintal_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 3
  df[columna_valor][mascara] = df[columna_valor][mascara] * 100 * 0.45359236996888618876
  return df

def conversion_kilogramo_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 2
  df[columna_valor][mascara] = df[columna_valor][mascara] * 1
  return df

def conversion_libra_litro(df, columna_unidades, columna_valor):
  mascara = df[columna_unidades] == 1
  df[columna_valor][mascara] = df[columna_valor][mascara] * 0.45359236996888618876
  return df

In [31]:
cacao_reps = conversion_tonelada_litro(cacao_reps, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')
cacao_reps = conversion_quintal_litro(cacao_reps, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')
cacao_reps = conversion_kilogramo_litro(cacao_reps, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')
cacao_reps = conversion_libra_litro(cacao_reps, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')

cacao = conversion_tonelada_litro(cacao, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')
cacao = conversion_quintal_litro(cacao, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')
cacao = conversion_kilogramo_litro(cacao, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')
cacao = conversion_libra_litro(cacao, 'cp_umed_npk_fq', 'cp_cantidad_npk_fq')

### FUNCIONES

In [32]:
def parametro_ventas(df_repetidos, df_unicos, col_superficie, col_ventas):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  df_unicos['parametros_ventas'] = 0
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    
    array_superficie_cos = df_repetidos[mascara][col_superficie].values
    array_superficie_cos = np.nan_to_num(array_superficie_cos, nan=0)
    superficie_cosechada = array_superficie_cos.sum()
    
    array_ventas = df_repetidos[mascara][col_ventas].values
    array_ventas = np.nan_to_num(array_ventas, nan=0)
    ventas = array_ventas.sum()

    division = ventas / superficie_cosechada
    df_unicos.loc[id, 'parametro_ventas'] = division
  
  df_unicos.reset_index(inplace=True)
  return df_unicos


def parametro_insumos(df_repetidos, df_unicos, col_superficie, col_insumos):
  ids = df_unicos['Identificador']
  df_unicos.set_index('Identificador', inplace=True)
  df_unicos['parametros_ventas'] = 0
  for id in ids:
    mascara = df_repetidos['Identificador'] == id
    
    array_superficie_plt = df_repetidos[mascara][col_superficie].values
    array_superficie_plt = np.nan_to_num(array_superficie_plt, nan=0)
    superficie_plantada = array_superficie_plt.sum()
    
    array_insumos = df_repetidos[mascara][col_insumos].values
    array_insumos = np.nan_to_num(array_insumos, nan=0)
    insumos = array_insumos.sum()

    division = insumos / superficie_plantada
    df_unicos.loc[id, 'parametro_insumos'] = division
  
  df_unicos.reset_index(inplace=True)
  return df_unicos

### cacao

In [33]:
# muestra = rd.sample(list(cacao_reps['Identificador'][cacao_reps['Identificador'].duplicated()]), 5)
muestra = [5035104000950017, 13045302156850002, 2075003000760015, 6105002000090001, 12115002017210006]
muestra

[5035104000950017,
 13045302156850002,
 2075003000760015,
 6105002000090001,
 12115002017210006]

In [34]:
cacao_reps[cacao_reps['Identificador'].isin(muestra)][['Identificador', 'cp_k411', 'cp_k409', 'ventas', 'cp_cantidad_npk_fq']]

,Identificador,cp_k411,cp_k409,ventas,cp_cantidad_npk_fq
294,6105002000090001,10.00,10.00,2.272727,453.59237
295,6105002000090001,2.00,2.00,2.272727,NaN
872,13045302156850002,2.00,2.00,1.363636,200.00000
873,13045302156850002,3.00,3.00,0.681818,100.00000
874,13045302156850002,0.98,0.98,0.681818,100.00000
1339,5035104000950017,1.00,1.50,0.227273,NaN
1340,5035104000950017,0.50,1.00,0.090909,NaN
1882,2075003000760015,3.00,3.00,4.090909,150.00000
1883,2075003000760015,7.00,7.00,9.545455,350.00000
2129,12115002017210006,0.50,0.50,0.681818,100.00000


In [35]:
cacao = parametro_ventas(cacao_reps, cacao, 'cp_k411', 'ventas')
cacao = parametro_insumos(cacao_reps, cacao, 'cp_k409', 'cp_cantidad_npk_fq')

In [36]:
cacao[cacao['Identificador'].isin(muestra)][['Identificador', 'cp_k411', 'cp_k409', 'ventas', 'parametro_insumos', 'parametro_ventas']]

,Identificador,cp_k411,cp_k409,ventas,parametro_insumos,parametro_ventas
262,6105002000090001,10.0,12.00,4.545455,37.799364,0.378788
720,13045302156850002,2.0,5.98,2.727273,66.889632,0.456066
1078,5035104000950017,1.0,2.50,0.318182,0.000000,0.212121
1487,2075003000760015,3.0,10.00,13.636364,50.000000,1.363636
1681,12115002017210006,0.5,4.74,8.500000,105.485232,1.793249


In [37]:
cacao['parametro_insumos'].replace([np.inf, -np.inf], np.nan, inplace=True)
cacao['parametro_ventas'].replace([np.inf, -np.inf], np.nan, inplace=True)
cacao[cacao['Identificador'].isin(muestra)][['Identificador', 'cp_k411', 'cp_k409', 'ventas', 'parametro_insumos', 'parametro_ventas']]

,Identificador,cp_k411,cp_k409,ventas,parametro_insumos,parametro_ventas
262,6105002000090001,10.0,12.00,4.545455,37.799364,0.378788
720,13045302156850002,2.0,5.98,2.727273,66.889632,0.456066
1078,5035104000950017,1.0,2.50,0.318182,0.000000,0.212121
1487,2075003000760015,3.0,10.00,13.636364,50.000000,1.363636
1681,12115002017210006,0.5,4.74,8.500000,105.485232,1.793249


### GUARDADO

In [38]:
cacao.to_csv(os.path.join(ruta_resultados, 'cacao.csv'), index=False, encoding='utf-8')

In [39]:
cacao.to_stata(os.path.join(ruta_resultados, 'cacao.dta'), write_index=False)